In [1]:
import folium
import pandas as pd

In [2]:
from folium.features import DivIcon

In [3]:
data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df = pd.read_csv(data_url)

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, launch_site_info in launch_sites_df.iterrows():
    
    class_series = spacex_df[spacex_df['Launch Site'] == launch_site_info["Launch Site"]]['class']
    
    number_of_successes = class_series[class_series == 1].count()
    number_of_failures = class_series[class_series == 0].count()
    
    circle = folium.Circle(
        [launch_site_info['Lat'], launch_site_info['Long']], 
        radius=1000, 
        color='#d35400', fill=True
    ).add_child(folium.Popup(f'Successes : {number_of_successes} - Failures :  {number_of_failures}'))
    
    marker = folium.map.Marker(
        [launch_site_info['Lat'], launch_site_info['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_info['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [ ]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster()

spacex_df['marker_color'] = spacex_df['class'].map(lambda x : 'green' if x == 1 else 'red')

In [ ]:
spacex_df.tail()

In [ ]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    marker = folium.map.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

In [ ]:
from folium.plugins import MousePosition

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map